In [1]:
import pandas as pd
import re
import numpy as np
from multiprocessing import  Pool

In [2]:
cvd_regional = pd.read_csv("../data/raw/covid/covid_geographical.csv")

/home/caligine/.conda/envs/eecs_731/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
cvd_regional.columns[5]

'location_geom'

In [4]:
cvd_regional.columns[11]

'admin2'

In [5]:
cvd_regional.columns

Index(['province_state', 'country_region', 'date', 'latitude', 'longitude',
       'location_geom', 'confirmed', 'deaths', 'recovered', 'active', 'fips',
       'admin2', 'combined_key'],
      dtype='object')

In [6]:
cvd_regional.loc[cvd_regional.country_region=='US'].province_state.unique()

array(['New York', 'Washington', 'California', 'New Jersey', 'Illinois',
       'Florida', 'Louisiana', 'Michigan', 'Massachusetts', 'Georgia',
       'Colorado', 'Texas', 'Pennsylvania', 'Connecticut', 'Wisconsin',
       'Tennessee', 'North Carolina', 'Ohio', 'Maryland', 'Virginia',
       'Nevada', 'Minnesota', 'Oregon', 'South Carolina', 'Utah',
       'Alabama', 'Arkansas', 'Indiana', 'Maine', 'Mississippi',
       'Diamond Princess', 'Arizona', 'Iowa', 'New Hampshire', 'Oklahoma',
       'Rhode Island', 'District of Columbia', 'Kentucky', 'New Mexico',
       'Kansas', 'Missouri', 'Delaware', 'Nebraska', 'Grand Princess',
       'Vermont', 'North Dakota', 'Wyoming', 'Hawaii', 'Guam', 'Idaho',
       'Montana', 'South Dakota', 'Alaska', 'Puerto Rico',
       'Virgin Islands', 'United States Virgin Islands', 'West Virginia',
       'US', 'Unassigned Location (From Diamond Princess)',
       'King County, WA', 'Santa Clara, CA', 'Westchester County, NY',
       'Snohomish County, WA

In [ ]:
us = cvd_regional.loc[cvd_regional.country_region=='US']

In [175]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [170]:
reg_re = re.compile(r"(((?P<cnt>[a-zA-Z-]+)\s(County|Parish),)|(?P<loc>[a-zA-Z-]+),)?\s(?P<st>[A-Z]{2})(?P<dp> \(From ?(Diamond|Grand) ?Princess\))?",)

In [186]:
def us_xform(df):
    c = pd.DataFrame()
    
    for k, v in df.iteritems():
        m = reg_re.match(v)
        s = pd.Series(name=k, dtype='object')
        s['cruise_ship'] = False
        s['city'] = None
        s['county'] = None
        if v in us_state_abbrev:
            s['state'] = us_state_abbrev[v]
        else:
            s['state'] = None
            
        if m:
            dp = m.group('dp') if m.group('dp') else None
            cnt = m.group('cnt') if m.group('cnt') else None
            state = m.group('st') if m.group('st') else None
            loc = m.group('loc') if m.group('loc') else None
            
            if dp:
                s['cruise_ship'] = True

            if cnt:
                s['county'] = cnt
            elif loc and not (loc == 'Unassigned Location'):
                s['city'] = loc

            if state and not (state=='Unassigned Location'):
                s['state'] = state
        c = c.append(s)
    return c

In [187]:
pool = Pool(16)
sp = np.array_split(us.province_state, 16)
sp = pool.map(us_xform, sp)
sp = pd.concat(sp)
pool.close()
pool.join()

In [192]:
s = us.merge(sp, left_index=True, right_index=True)

In [196]:
s = s[['date', 'confirmed', 'deaths', 'recovered', 'city', 'county', 'cruise_ship', 'state']]

In [197]:
s.to_csv("../data/processed/covid/us_regional_covid.csv")